<a href="https://colab.research.google.com/github/HenriqueCF8896/MNCM/blob/main/Programa4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
bairstow_fractal.py

Implementação do método de Bairstow para encontrar raízes de polinômios
com geração do fractal de Bairstow.

Saídas:
  - Imprime as raízes encontradas.
  - Gera o arquivo `dados_bairstow_simulado.txt` com colunas (r0, s0, iterações).
  - Cria um script Gnuplot `fractal_bairstow.gnu` com configurações avançadas para plotagem.

Uso:
  python3 bairstow_fractal.py

Dependências:
  - numpy
"""
import numpy as np

# Função principal que aplica o método de Bairstow
# a: coeficientes do polinômio (a0, a1, ..., an)
# r, s: palpites iniciais para o fator quadrático x^2 - r x - s
# tol: tolerância de convergência
# max_iter: iterações máximas
# Retorna: (raízes encontradas, iterações na última deflação)
def bairstow(a, r=1.0, s=1.0, tol=1e-6, max_iter=1000):
    # Copia coeficientes e configura variáveis iniciais
    a_work = np.array(a, dtype=float)
    roots = []            # lista de raízes encontradas
    it_count = 0          # contador de iterações da última etapa

    deg = a_work.size - 1  # grau atual do polinômio
    # Enquanto o polinômio tiver grau > 2, extrai fatores quadráticos
    while deg > 2:
        rr, ss = r, s     # cópias dos palpites para r e s
        it = 0            # contador de iterações deste ciclo

        # Loop de Newton-Raphson para ajustar rr e ss
        while it < max_iter:
            # Divisão sintética: calcula b e derivada sintética c
            b = np.zeros(deg+1)
            c = np.zeros(deg+1)
            b[deg] = a_work[deg]
            b[deg-1] = a_work[deg-1] + rr * b[deg]
            # Preenche coeficientes b
            for i in range(deg-2, -1, -1):
                b[i] = a_work[i] + rr * b[i+1] + ss * b[i+2]
            # Preenche c (derivadas de b)
            c[deg] = b[deg]
            c[deg-1] = b[deg-1] + rr * c[deg]
            for i in range(deg-2, 0, -1):
                c[i] = b[i] + rr * c[i+1] + ss * c[i+2]

            # Cálculo das correções dr e ds
            D = c[2]*c[2] - c[3]*c[1]
            if abs(D) < tol:
                break
            dr = (-b[1]*c[2] + b[0]*c[3]) / D
            ds = (-b[0]*c[2] + b[1]*c[1]) / D
            rr += dr
            ss += ds
            it += 1
            # Convergência: verifica se mudanças são menores que tol
            if abs(dr) < tol and abs(ds) < tol:
                break

        # Após convergir, calcula raízes do fator quadrático encontrado
        disc = rr*rr + 4*ss
        root1 = (rr + np.sqrt(disc)) / 2
        root2 = (rr - np.sqrt(disc)) / 2
        roots.extend([root1, root2])
        it_count = it

        # Deflação: remove o fator quadrático do polinômio
        a_work = b[2:].copy()
        deg = a_work.size - 1

    # Trata casos de grau 2 ou 1 restantes sem iteração
    if deg == 2:
        # Equação quadrática direta
        a0, a1, a2 = a_work
        disc = a1*a1 - 4*a2*a0
        root1 = (-a1 + np.sqrt(disc)) / (2*a2)
        root2 = (-a1 - np.sqrt(disc)) / (2*a2)
        roots.extend([root1, root2])
    elif deg == 1:
        # Equação linear direta
        roots.append(-a_work[0] / a_work[1])

    return np.array(roots), it_count

# Função que gera dados para o fractal de Bairstow
# a: coeficientes do polinômio
# r_min, r_max, s_min, s_max: intervalo dos palpites iniciais
# grid_size: resolução da malha
# Retorna lista de tuplas (r0, s0, iterações)
def fractal_data(a, r_min, r_max, s_min, s_max, grid_size=400, tol=1e-6, max_iter=999):
    a = np.array(a, dtype=float)
    data = []
    # Varre toda a malha de valores iniciais r0 e s0
    for r0 in np.linspace(r_min, r_max, grid_size):
        for s0 in np.linspace(s_min, s_max, grid_size):
            _, it = bairstow(a, r0, s0, tol, max_iter)
            data.append((r0, s0, it))
    return data

# Ponto de entrada do script
def main():
    # Leitura dos coeficientes via input do usuário
    try:
        entrada = input(
            "Digite os coeficientes do polinômio (termo independente ao maior grau), separados por espaço:\n"
        ).strip()
        a = [float(x) for x in entrada.split()]
    except Exception as e:
        print(f"Erro ao ler os coeficientes: {e}")
        return

    # Chute inicial padrão para Bairstow
    r0, s0 = 0.1, 0.1
    # Executa o método e imprime resultados
    roots, iterations = bairstow(a, r0, s0)
    print(f"\nRaízes encontradas: {roots}")
    print(f"Iterações no último fator: {iterations}")

    # Geração dos dados do fractal e salvamento em arquivo
    data = fractal_data(a, -3, 3, -3, 3, grid_size=400)
    with open("dados_bairstow_simulado.txt", "w") as f:
        for r, s, it in data:
            f.write(f"{r}\t{s}\t{it}\n")
    print("Dados do fractal salvos em dados_bairstow_simulado.txt")

    # Monta e salva o script Gnuplot com estilo avançado
    gnuplot_script = """
set terminal pngcairo size 1000,800 enhanced font 'Arial,10'
set output 'mapa_bairstow.png'

set title "Fractal de Bairstow - Número de Iterações até Convergência"
set xlabel "r_0 (chute inicial)"
set ylabel "s_0 (chute inicial)"
set cblabel "Número de Iterações"

set palette defined ( \
  0 "#440154", \
  1 "#482777", \
  2 "#3E4989", \
  3 "#31688E", \
  4 "#26828E", \
  5 "#1F9E89", \
  6 "#35B779", \
  7 "#6CCE59", \
  8 "#B4DE2C", \
  9 "#FDE725" \
)

set view map
set datafile missing "999"

# Plotagem utilizando image sem título
splot 'dados_bairstow_simulado.txt' using 1:2:3 with image notitle
"""
    with open("fractal_bairstow.gnu", "w") as f:
        f.write(gnuplot_script)
    print("Script Gnuplot salvo em fractal_bairstow.gnu")

# Execução quando chamado diretamente
if __name__ == "__main__":
    main()


Digite os coeficientes do polinômio (termo independente ao maior grau), separados por espaço:
1.25 -3.875 2.125 2.75 -3.5 1


<ipython-input-1-f6923108d2cf>:70: RuntimeWarning: invalid value encountered in sqrt
  root1 = (rr + np.sqrt(disc)) / 2
<ipython-input-1-f6923108d2cf>:71: RuntimeWarning: invalid value encountered in sqrt
  root2 = (rr - np.sqrt(disc)) / 2



Raízes encontradas: [ 2.         -1.                 nan         nan  0.50000038]
Iterações no último fator: 12
Dados do fractal salvos em dados_bairstow_simulado.txt
Script Gnuplot salvo em fractal_bairstow.gnu
